# LangChain RAG with HANA Vector Store and Optional Chat History

This notebook walks you through building a **Retrieval-Augmented Generation (RAG)** application using:
- **LangChain LCEL**
- **HANA Vector Store**
- **OpenAI GPT-4o**
- And an **optional chat history feature**.


## Prerequisites

* **HANA Vector Store**:

    The HANA Vector Store is pre-populated with text documents, their corresponding embedding vectors, and associated metadata.

    For this tutorial, the necessary source documents have already been processed and stored in the Vector Store.

    To view the **source PDF documents** and learn how to prepare and store data in the HANA Vector Store—including embedding generation and metadata handling — refer [Embeddings Best Practices](https://github.com/SAP-samples/sap-btp-ai-best-practices/tree/main/best-practices/vector-rag-embedding) guide.


## Setup

### Dependencies
Ensure all python packages provided in `requirements.txt` file are installed.
``` bash
pip install -r requirements.txt
```


### Environment Variables
Ensure `.env` file is updated with following environment variables.
``` env
HANA_ADDRESS=your_hana_host
HANA_PORT=your_port
HANA_USER=your_user
HANA_PASSWORD=your_password
HANA_AUTOCOMMIT=true
HANA_SSL_CERT_VALIDATE=false

AICORE_AUTH_URL=your_aicore_auth_url
AICORE_CLIENT_ID=your_aicore_client_id
AICORE_CLIENT_SECRET=your_aicore_secret
AICORE_RESOURCE_GROUP=your_resource_group
AICORE_BASE_URL=your_base_url
```


## Initialization and Imports

We begin by importing necessary components and setting up the environment.


In [1]:
# Import section

import os
from dotenv import load_dotenv
load_dotenv(override=True)

from hdbcli import dbapi

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.vectorstores import HanaDB
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
def str_to_bool(value: str) -> bool:
    '''
    Convert environment variable to boolean based on values.
    '''
    return value.lower() in ("true", "1", "yes")

## Set Up HANA Vector Store

In [5]:
# Establish connection to HANA Vector Store
connection = dbapi.connect(
    os.environ["HANA_ADDRESS"],
    os.environ["HANA_PORT"],
    os.environ["HANA_USER"],
    os.environ["HANA_PASSWORD"],
    str_to_bool(os.getenv("HANA_AUTOCOMMIT", "false")),
    str_to_bool(os.getenv("HANA_SSL_CERT_VALIDATE", "false"))
)

# Establish connection to Generative AI Hub on AI Core for LLM access
proxy_client = get_proxy_client("gen-ai-hub")

# Initialize embeddings model
embedding_model = OpenAIEmbeddings(proxy_model_name="text-embedding-ada-002")

# Connect to HANA Vector Store and create LangChain object of the vector store for further operations
vdb = HanaDB(
    embedding=embedding_model,
    connection=connection,
    distance_strategy=DistanceStrategy.COSINE, # Use cosine similarity for vector search
    table_name="SAP_HELP_PUBLIC",
    
    # The following column names are the expected defaults used by HanaDB:
    content_column="VEC_TEXT",     # (default) Column containing the raw document text
    metadata_column="VEC_META",    # (default) Column containing metadata (typically JSON)
    vector_column="VEC_VECTOR",    # (default) Column containing embedding vectors

    # If your table uses different column names, specify them here to override the defaults
)

# Set up the vector store as a retriever with top-k document retrieval
retriever = vdb.as_retriever(search_kwargs={"k": 2}) # Retrieve top 2 most similar documents

## Prompt Template

In [6]:
# Define prompt
user_prompt = '''
Use the following context information to answer to user's query.
Here is some context: {context}

Based on the above context, answer the following query:
{question}

The answer tone has to be very professional in nature.

If you don't know the answer, politely say that you don't know.
'''

# Create langchain prompt object to fit into langchain pipeline
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("user", user_prompt)
])

## Chat History (Optional)

We implement a capped in-memory message history. You can toggle storing this history using a flag.

In [7]:
# Necessary imports for processing conversation history messages
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """
    A class with methods add/get/delete operations on history data.
    """
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_message(self, message: BaseMessage) -> None:
        self.messages.append(message)
        self.messages = self.messages[-4:]

    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)
        self.messages = self.messages[-4:]

    def get_messages(self) -> List[BaseMessage]:
        return self.messages

    def clear(self) -> None:
        self.messages = []

store = {} # Global dictionary to store history messages in memory

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """
    Retrieve history messages for given session id.
    """
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

## Define the LLM and RAG Chain

In [8]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

# Initialize LLM from Generative AI Hub. For now using OpenAI model. Other models are supported as well as mentioned on help.sap.com
llm = ChatOpenAI(
    proxy_model_name='gpt-4o', 
    proxy_client=proxy_client,
    max_tokens=2000,
    temperature=0.5
)

# Define langchain pipeline chain from taking a question till output parsing
base_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
        "history": lambda x: x.get("history", [])
    }
    | prompt
    | llm
    | StrOutputParser()
)

## Add Optional Message History

In [9]:
# Wrap the chain with history message processing runnable to add history messages to conversation if opted.
chain_with_history = RunnableWithMessageHistory(
    base_chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

## Chat Function with History Toggle

In [11]:
def chat_with_rag(question: str, session_id: str, use_history: bool = True):
    """
    Function to decide if history messages to be processed based user's input
    """
    if use_history:
        return chain_with_history.invoke(
            {"question": question},
            config={"configurable": {"session_id": session_id}}
        )
    else:
        return base_chain.invoke({"question": question, "history": []})

## Run It!

In [12]:
if __name__ == "__main__":
    session_id = "user-123"
    print(chat_with_rag("What is Business AI and can you summarize the context you received?", session_id, use_history=True))
    print()
    print("-+-" * 30)
    print()
    print(chat_with_rag("Can you repeat?", session_id, use_history=True))

    # Without history
    print()
    print("-+-" * 30)
    print()
    print(chat_with_rag("What was your previous answer?", session_id, use_history=False))

Business AI refers to the application of artificial intelligence technologies within business environments to drive real business results. It encompasses various aspects such as facilitating human-machine interaction through natural user experiences, enabling automation for repetitive tasks, and augmenting human decisions and cognition with insights, optimization, and predictions. The context provided highlights the role of AI within a business technology platform, emphasizing its responsibility, reliability, and relevance.

The documents indicate that customers seeking to adopt Business AI are primarily motivated by the desire to achieve tangible business results. The AI foundation on the business technology platform supports this by enhancing user experience, automating processes, and providing valuable insights for decision-making.

If you have any further questions or need more detailed information, please feel free to ask.

-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-

## Summary

- This setup uses **SAP HANA Vector Store** for retrieval.
- Responses are generated using **GPT-4o** from OpenAI available on Generative AI Hub. Other models can be used as well as listed on [help.sap.com](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/supported-models)
- **Chat history** is optional and supports follow-up questions. Current implementation processes history messages in-memory. However, the same can be stored in files or Database as well. Please refer [LangChain document](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) for processing history messages.
- Implemented using **LangChain Expression Language (LCEL)** for composability.